In [ ]:
import re
import json

In [ ]:
from app.uzt_client import UZTClient
from app import pydantic_models as pd
import main

In [ ]:
#getting categories list
def get_categories_from_page(client):
    categories = list()
    a_list = client.tree.css("#ctl00_MainArea_UpdatePanel1 li a")
    for a in a_list:
        category = pd.Category(
            name = a.text(),
            href = a.attrs.get('href')
        )
        categories.append(category)
    
    return categories
    

In [ ]:
#getting href to next page
def get_next_page_href(client):
    a = client.tree.css_first("table.Pager td:last-child a")
    if not a:
        return False
    return a.attrs.get("href")


In [ ]:
uzt = UZTClient()
uzt.get(url="https://portal.uzt.lt/LDBPortal/Pages/ServicesForEmployees.aspx")

In [ ]:
cats = main.get_categories()

In [ ]:
cats

In [ ]:
create_resp = main.send_data_to_api(endpoint="/api/categories/create", data=cats.data)

In [ ]:
uzt.redirect_response.text

In [ ]:
uzt.response.text

In [24]:
a = uzt.tree.css_first("a")
a.attrs.get('href')

'#SPContainer'

In [ ]:
json.dumps(cats.data)

In [ ]:
#open category
uzt.submit_asp_form(cats[0].href)
uzt.get(url=uzt.next_url)

In [ ]:
uzt.save_to_html("save00_card.html")

In [ ]:
next_page_href = main.get_next_page_href(uzt)
print(next_page_href)

In [ ]:
#next page
uzt.submit_asp_form(next_page_href.data)


In [ ]:
# open page 2
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl29$PageBtn10','')")
uzt.save_to_html("save00_card.html")

In [ ]:
need_to_get = ("__VIEWSTATE", "__VIEWSTATEGENERATOR", "__EVENTVALIDATION")
asp_inputs = uzt.asp_form_inputs
asp_inputs

In [ ]:
uzt.get(url=uzt.next_url)

In [ ]:
jobs_list = list()

In [ ]:
# поиск строк и ячеек
table = uzt.tree.css_first("#ctl00_MainArea_SearchResultsList_POGrid")
tr_list = table.css("tr:not(:nth-child(-n+2)):not(:last-child)")
for tr in tr_list:
    cells = tr.css("td")
    date_from, date_to, title, company, place = [item.text().strip() for item in tr.css("td")]
    js_url = cells[2].css_first("a").attrs.get("href")
    uzt.submit_asp_form(js_url, only_url=True)
    dirty_url = uzt.next_url
    match = re.search(r"(^.+aspx\?).+(itemID.+)$", dirty_url)
    url = match.group(1)+ match.group(2)
    job = pd.Job(date_from=date_from,
              date_to= date_to,
              title= title,
              company= company,
              place= place,
              url = url,
             )
    
    jobs_list.append(job.dict())
    
print(len(jobs_list))
print(json.dumps(jobs_list))
cells
job


In [ ]:
# open job's card
uzt.submit_asp_form("javascript:__doPostBack('ctl00$MainArea$SearchResultsList$POGrid$ctl04$PositionTitleButton','')", only_url=True)

In [ ]:
uzt.next_url

In [ ]:
uzt.response.text

In [ ]:
uzt.asp_form_inputs


In [ ]:
class AttrFromDict(dict):
    def as_class(self, dictionary: dict):
        [setattr(self, key, value) for key, value in dictionary.items()]


In [ ]:
d = {"date_from": "2023-05-08", "date_to": "2023-05-19", "title": "\u016akio darbininkas", "company": "\u016akininko \u016bkis, 1359663", "place": "Mol\u0117t\u0173 rajono sav., Kreivi\u0161kiai", "url": "https://portal.uzt.lt//LDBPortal/Pages/PositionOpeningPublic/PublicOpenPositionCard.aspx?itemID=b70f28b2-89e0-4608-acfd-763670ca62b6"}
